In [ ]:
#In this exercise we chose the first use case which is "Predict the stock market price for Tesla. 
"""
We decided to use Regression Algorithm since it`s suitable for predictions of continious values such us price, age, weather, traffic etc. 
based on earlier results or measurements. On the other hand the classification algorithm is more suitable to classify or distict values as 
True or False, White or Black, Real or Fake and so on. 
"""